In [1]:
import os
os.chdir("../")

In [2]:
%pwd

'c:\\Users\\bohra\\Mnist_fashion_model'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    param_classes: int

In [4]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        params = self.params 
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            param_classes = params.classes
        )

        return prepare_base_model_config
    


In [5]:
import os
import urllib.request as request
from zipfile import ZipFile
import torch.nn.functional as F
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision

class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
    
    def get_base_model(self):
        self.model = torchvision.models.resnet18(pretrained=True)
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, self.config.param_classes)

        self.save_model(path=self.config.base_model_path, model=self.model)
    
    @staticmethod
    def _prepare_full_model(classes,model, freeze_all, freeze_till,):
        if freeze_all:
            for param in model.parameters():
                param.requires_grad = False
        elif freeze_till is not None and freeze_till > 0:
            for param in list(model.parameters())[:-freeze_till]:
                param.requires_grad = False

        # Modify the final layer for the number of classes
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, classes)

        return model
    
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,           
            freeze_all=True,
            freeze_till=None,
            classes=self.config.param_classes,

        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)
    
    @staticmethod
    def save_model(path: Path, model: nn.Module):
        torch.save(model.state_dict(), path)



In [6]:

try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

[2024-07-21 19:14:07,480: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-21 19:14:07,480: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-21 19:14:07,480: INFO: common: created directory at: artifacts]
[2024-07-21 19:14:07,491: INFO: common: created directory at: artifacts/prepare_base_model]


c:\Users\bohra\anaconda3\envs\honey_mnist\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\bohra\anaconda3\envs\honey_mnist\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
